In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.data.features import feature_engineering
from src.data.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.evaluator import evaluation

c:\Users\gonpr\ML_Projects\Ranking\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

In [5]:
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

In [6]:
import pandas as pd
from src.models.candidate import candidate_generation

df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train = pd.concat([df_train.iloc[:,:3], df_train_neg], ignore_index=True)

del df_train_neg

df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_valid)
    ]

In [7]:
# perform bayesian search
searcher = BayesianSearch(config['optimization']['hyper_params'], algorithm='XGBRanker')

def objective(trial) -> float:
    return searcher.fit(df_train, df_valid, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-04-22 17:07:44,938] A new study created in memory with name: no-name-d4ff19a1-dcbe-462c-bac2-b1fa72a4761c
[I 2025-04-22 17:07:46,652] Trial 0 finished with value: 0.9476401245128002 and parameters: {'learning_rate': 0.13349069140524963, 'gamma': 4.711351684265544, 'max_depth': 8, 'subsample': 0.6127362974517998, 'n_estimators': 134}. Best is trial 0 with value: 0.9476401245128002.
[I 2025-04-22 17:07:50,184] Trial 1 finished with value: 0.9454234372048969 and parameters: {'learning_rate': 0.029229768798753476, 'gamma': 0.0904263064921984, 'max_depth': 7, 'subsample': 0.8163199609056089, 'n_estimators': 240}. Best is trial 0 with value: 0.9476401245128002.
[I 2025-04-22 17:07:53,182] Trial 2 finished with value: 0.9474871960760828 and parameters: {'learning_rate': 0.12808340444892047, 'gamma': 4.48086632682986, 'max_depth': 8, 'subsample': 0.9884290017913665, 'n_estimators': 289}. Best is trial 0 with value: 0.9476401245128002.
[I 2025-04-22 17:07:55,051] Trial 3 finished with v

In [8]:
print("Best trial:")
print(study.best_trial.params)

Best trial:
{'learning_rate': 0.13031965305132584, 'gamma': 2.4746054762088905, 'max_depth': 3, 'subsample': 0.7104877097542801, 'n_estimators': 99}


In [9]:
import pandas as pd

# use indicator to know the origin
df_train = dfs['data'].merge(
    df_test
    , on=['user_id', 'item_id', 'rating'], how='left'
    , indicator=True
    )
# keep only rows that are present in df1 but not in df2
df_train = df_train[df_train['_merge'] == 'left_only'].drop(columns=['_merge'])

user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative sampling
df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train = pd.concat([df_train, df_train_neg], ignore_index=True)

del df_train_neg

df_train, df_test = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_test)
    ]

In [10]:
from sklearn.metrics import ndcg_score
from xgboost import XGBRanker
import numpy as np

clf = XGBRanker(**study.best_trial.params)
clf.fit(
    df_train['X'], df_train['y'].astype(int)
    , group=df_train['group']
    , verbose=False
    )

preds = clf.predict(df_test['X'])

print(evaluation(df_test['y'], preds, df_test['group']))

0.9531453590722662


In [11]:
from src.models.baseline import baseline_model

result = baseline_model(dataframes=dfs, n=10)
group = result.groupby('user_id').size().to_list()

print(evaluation(result['rating'], result['est_rating'], group))
del group

0.9168812846621082


In [12]:
user_item_features = feature_engineering(dataframes={
    'user': dfs['user'], 'item': dfs['item'], 'data': dfs['data']
    })

df_baseline = build_rank_input(ratings=result.drop(columns=['est_rating']), features=user_item_features)

preds = clf.predict(df_baseline['X'])
print(evaluation(df_baseline['y'], preds, df_baseline['group']))

0.9499964153201759
